In [18]:
import gdown, zipfile, os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

print('Dependencies loaded\n')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if not str(device) == 'cuda':
  raise SystemError('GPU device not found')
print('Found GPU: {}'.format(device))

Dependencies loaded

Found GPU: cuda


In [ ]:
#Dataset Download
output='dataset.zip'
id= None    # to be defined 

dataset_url = 'https://drive.google.com/uc?id=' + id + '&export=download&confirm=t'
gdown.download(dataset_url, output, quiet=False)

# Dataset extracting
zip_file_path = None  # the path to the zip file downloaded before
destination_folder = None # the destination folder

print('Extracting the zipped dataset')

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
     zip_ref.extractall(destination_folder)
print('Deleting the previous downloaded zip')
os.remove('dataset.zip')
print('--Deleted')

In [2]:
model = models.resnet50(pretrained=True)

# Freeze pre-trained layers
for param in model.parameters():
    param.requires_grad = False

# Replace the last fully connected layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # 2 for binary classification


c:\Users\gianc\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\gianc\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)  # Optimizer for training the last layer

In [ ]:
# Define data transformations
transformTrain = transforms.Compose([
    transforms.RandomHorizontalFlip(0.5),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transformsTest = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define paths for train_dir and test_dir according to split perfomed on OFs dataset
train_dir, test_dir = None

# Load the custom dataset
train_dataset = ImageFolder(root=train_dir, transform=transformTrain)
test_dataset = ImageFolder(root=test_dir, transform=transformsTest)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model = model.to(device)

num_epochs = 10
for epoch in range(num_epochs):
    # Training loop
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print training progress
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

